# Day 5: Hydrothermal Venture

[*Advent of Code 2021 day 5*](https://adventofcode.com/2021/day/4) and [*solution megathread*](https://www.reddit.com/r9824c)

[![nbviewer](https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.svg)](https://nbviewer.jupyter.org/github/UncleCJ/advent-of-code/blob/master/2021/05/code.ipynb) [![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/UncleCJ/advent-of-code/master?filepath=2021%2F05%2Fcode.ipynb)

In [1]:
from IPython.display import HTML
import sys

sys.path.append('../../')
import common

downloaded = common.refresh()
%store downloaded >downloaded

Writing 'downloaded' (dict) to file 'downloaded'.


## Part One

In [2]:
HTML(downloaded['part1'])

## Boilerplate

Let's try using [pycodestyle_magic](https://github.com/mattijn/pycodestyle_magic) with pycodestyle (flake8 stopped working for me in VS Code Jupyter). Now how does type checking work?

In [3]:
%load_ext pycodestyle_magic

In [4]:
%pycodestyle_on

## Comments

...

In [5]:
testdata = """0,9 -> 5,9
8,0 -> 0,8
9,4 -> 3,4
2,2 -> 2,1
7,0 -> 7,4
6,4 -> 2,0
0,9 -> 2,9
3,4 -> 1,4
0,0 -> 8,8
5,5 -> 8,2""".splitlines()

# inputdata = downloaded['input'].splitlines()

In [6]:
def my_part1_solution(data: list[str],
                      debug: bool = False) -> int:
    return 0

In [7]:
# assert(my_part1_solution(testdata, debug=True) == 5)

In [8]:
# my_part1_solution(inputdata)

In [9]:
HTML(downloaded['part1_footer'])

## Part Two

In [10]:
# HTML(downloaded['part2'])

In [11]:

def my_part2_solution(data: list[str],
                      debug: bool = False) -> int:
    return 0

In [12]:
# assert(my_part2_solution(testdata, debug=True) == ...)

In [13]:
# my_part2_solution(inputdata)

In [14]:
# HTML(downloaded['part2_footer'])